# Module 1

## Team Members:
Max Calcoen

Medha Tadavarthi

## Project Title:
*(Fill in)*



## Project Goal:
This project seeks to... *(what is the purpose of your project -- i.e., describe the question that you seek to answer by analyzing data.)*

## Disease Background: 
*Fill in information about 11 bullets:*

* Prevalence & incidence
* Economic burden
* Risk factors (genetic, lifestyle) 
* Societal determinants
* Symptoms

* Diagnosis ([source](https://www.alz.org/alzheimers-dementia/diagnosis/medical_tests))
    * no single test to evaluate alzheimer's or other dementia
    * medical history: history of cognitive and behavioral changes, family history of alzheimer's or other dementia
    * physical exam, diagnostic tests: diet, medications, physicals, lab test
    * neurological exam: reflexes, coordination, eye movement, speech, sensation
    * cognitive, functional and behavioral tests: AD8, SLUMS, IQCODE, FAQ, MMSE
    * computerized tests and devices: growing area. includes ANAM, CANTAB, CognICA, Cognigram, Cognivue, Cognision (digital cognitive testing tools)
    * depression screen, mood assessment: sense of wellbeing
    * genetic testing: certain genes increase risk of alzheimer's and other rare genes (1% or less) that directly cause alzheimer's
    * brain imaging: MRI or CT scanes rule out other conditions, or sometimes look directly at beta-amyloid
    * cerebrospinal fluid: biomarkers (tau, beta-amyloid, neurofilament light) appear in CSF before brain, still evolving test
    * blood test: evolving method
* Standard of care treatments (& reimbursement) ([source](https://www.alz.org/alzheimers-dementia/treatments/navigating-treatment-options))
    * no cure, but various drug and non-drug options for treatment
    * drugs that treat symptoms: cognitive symptoms (chrlinesterase inhibitors, glutamate regulators), non-cognitive symptoms (orexin inhibitors), atypical antipsychotics (target serotonin and dopamine chemical pathways)
    * drugs that change disease progression: 2 FDA-approved treatments (both anti-amyloid antibody IV infusions)
    * as with many neurodegenerative disease, early diagnosis is important (some treatments only effective in early stages of disease). AD is a progressive brain disease: worsens over time (asymptomatic -> mild cognitive impairment -> mild -> moderate -> severe)
* Disease progression & prognosis ([source](https://www.alz.org/alzheimers-dementia/stages))
    * 3 stages: early (mild), middle (moderate), and late (severe)
    * on average, a person lives 4-8 years after diagnosis
    * early stage: may function independently. feel as if they have memory lapses (forgetting words, familiar objects, names, etc)
    * middle stage: longest stage, lasts many years. dementia symptoms progress, including memory loss, confusion, trouble controlling bladder/bowels, personality / behavioral changes
    * late stage: lose ability to respond to environment, control movement, communication becomes difficult. require assistance with daily personal care, lose awareness, difficulty communicating
* Continuum of care providers ([source](https://aspe.hhs.gov/national-plan-address-alzheimers-disease))
    * depends on location (country > state > region > city)
    * primary and specialized medical providers provide timely and accurate diagnosis, neurologists, psychiatrists, neuropsychologists offer early detection and specialized treatment services
    * care coordination and transition services: care coordinators oversee transitions between stages of care (diagnosis to long term management)
    * long term support and residential care: assisted-living facilities, continuing care retirement communities, nursing homes (hopefully with dementia-specific protocols)
    * support for caregivers and families (educational materials, culturally sensitive support, community based programs)
    * public health and policy level 
* Biological mechanisms (anatomy, organ physiology, cell & molecular physiology)
* Clinical Trials/next-gen therapies

## Data-Set: 
*(Describe the data set(s) you will analyze. Cite the source(s) of the data.)*

## Data Analyis: 
*(Describe how you analyzed the data. This is where you should intersperse your Python code so that anyone reading this can run your code to perform the analysis that you did, generate your figures, etc.)*

## Verify and validate your analysis: 
*(Describe how you checked to see that your analysis gave you an answer that you believe (verify). Describe how your determined if your analysis gave you an answer that is supported by other evidence (e.g., a published paper).*

## Conclusions and Ethical Implications: 
*(Think about the answer your analysis generated, draw conclusions related to your overarching question, and discuss the ethical implications of your conclusions.*

## Limitations and Future Work: 
*(Think about the answer your analysis generated, draw conclusions related to your overarching question, and discuss the ethical implications of your conclusions.*